In [ ]:
import numpy as np
import geopandas as gpd
from scipy.spatial import cKDTree
from datetime import datetime 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
import pandas as pd
import contextily as cnx
import matplotlib.pyplot as plt
from datetime import date
import re
from rasterstats import zonal_stats
import rasterio

<h2> Obtaining fire dates and information for each Footprint <h2>

In [ ]:
#Reading all the necessary files

#Arc of Deforestation outline
gf = gpd.read_file('AoD.geojson')

#Fire scars (raster). Years-> Bands ,  Month -> Pixel value (0=no fire)
fire_scars = rasterio.open(PUT RASTER HERE)

#GEDI shots in the fire areas 
# (Already filtered -> Sensitivity>0.98 , Degraded filter, Quality filter, Surface filter and only Full beams)
GEDI_shots = gpd.read_file(Gedi_fire_fishnet_test.gpkg)

In [ ]:
# Get bounding box coordinates
minx, miny, maxx, maxy = gf.total_bounds

# Define grid size (adjust depending on CRS: degrees for EPSG:4326, meters for projected) 
cell_width = 0.5 # change this to control grid size 
cell_height = 0.5

# Generate rows and columns 
cols = np.arange(minx, maxx, cell_width) 
rows = np.arange(miny, maxy, cell_height) 

# Create polygons for each grid cell 
grid_cells = [] 
for x in cols: 
    for y in rows: 
        cell = box(x, y, x + cell_width, y + cell_height) 
        grid_cells.append(cell) 
        
# Convert to GeoDataFrame 
fishnet = gpd.GeoDataFrame({'geometry': grid_cells}, crs=gf.crs).drop_duplicates('geometry') 

fishnet = gpd.sjoin(fishnet, gf, how='inner').drop_duplicates('geometry') 

fishnet['tile_id']=fishnet.index

#ax = fishnet.boundary.plot() 
#sample = fishnet.iloc[[23]] 
#sample.geometry.plot(color='red', edgecolor='black', ax = ax, alpha=0.5) 
#gf.plot(ax=ax, color = 'yellow', alpha = 0.2) 

fishnet.explore()

In [ ]:
#Define function
def extract_fire_dates(fire_scars, GEDI_shots):        
    #Define de name the raster (fire_scars) to start the loop
    with fire_scars as src:                             
        #Ensure GEDI shots are in the same projection as fire data
        gdf_polygons = GEDI_shots.to_crs(src.crs)      
        #Gets the band year
        band_years = [int(re.search(r"(\d{4})", d).group(1)) for d in src.descriptions]   

        all_fire_dates = []
        fire_counts = []

        for poly in gdf_polygons.geometry:
            #Get the zonal stats for each polygon about fire occurence. 
            #It loops over the bands to get fire dates inside the polygon for each year (bands)
            stats = zonal_stats(poly, raster_path, stats="majority", band=list(range(1, src.count+1)))    
            dates = []
            #b -> band index (position in the list),  s-> stats for that band (mode of fire pixels inside the footpint - burn month)
            for b, s in enumerate(stats):                  
                month = s["majority"]
                #if month=0 -> no fire / if month>0, store the date (month and year)
                if month > 0:                              
                    year = band_years[b]
                    dates.append(date(year, month, 1))
            all_fire_dates.append(dates)
            fire_counts.append(len(dates))
        
        #Store all the fire dates
        gdf_polygons["fire_dates"] = all_fire_dates
        #Store the count of fire in the period
        gdf_polygons["fire_count"] = fire_counts
        #Store the first fire d[0]
        gdf_polygons["first_fire_date"] = [d[0] if d else None for d in all_fire_dates]
        #Store the last fire d[-1]
        gdf_polygons["last_fire_date"] = [d[-1] if d else None for d in all_fire_dates]

        return gdf_polygons

<h2> Pair matching <h2>

<h4> Finding pairs at a max. distance of 40m and with fire occurence in between them <h4>

In [ ]:
def get_close_pairs_with_fire(gdf_proj, max_distance=40):

    #Coordinates from the footprints
    coords = np.array([gdf_proj.geometry.x.values, gdf_proj.geometry.y.values])

    #creating the KD tree
    tree = cKDTree(coords.T)

    #Distance matrix to get distances (with a max. 40m)
    dist_matrix = tree.sparse_distance_matrix(tree, max_distance=max_distance, output_type='coo_matrix')

    #Get the near-coincident footprints
    close_pairs = [(i, j, d) for i, j, d in zip(dist_matrix.row, dist_matrix.col, dist_matrix.data) if i < j]

    #Record the pairs information 
    
    records = []
    for i, j, d in close_pairs:
        fi = gdf_proj.iloc[i]
        fj = gdf_proj.iloc[j]

        # Observation times
        t_i = fi.time
        t_j = fj.time

        # Fire dates for footprint j
        fire_dates = fj.fire_dates

        # Fire flag: True if any fire occurred between times
        fire = any((t_i < fd <= t_j) for fd in fire_dates)

        # Post-fire time in months
        post_fire_months = None
        if fire:
            fire_date = min([fd for fd in fire_dates if t_i < fd <= t_j])
            post_fire_months = (t_j.year - fire_date.year) * 12 + (t_j.month - fire_date.month)

        records.append({
            "index_1": i,
            "index_2": j,
            "distance": d,
            "time_1": t_i,
            "time_2": t_j,
            "fire": fire,
            "post_fire_months": post_fire_months,
            "agb_1": fi.agb,   # biomass from footprint 1
            "agb_2": fj.agb,   # biomass from footprint 2
            "delta_agb": fi.agb - fj.agb,  # biomass difference
            "geometry_1": fi.geometry,
            "geometry_2": fj.geometry
        })

    pairs_gdf = gpd.GeoDataFrame(records, geometry="geometry_2").sort_values("distance")
    return pairs_gdf